Importieren grundlegender Packages

In [2]:
# Dataframes und Datenanalyse-Funktionen
import pandas as pd

# Numerische Berechnungen mit Dataframes/Listen etc. 
import numpy as np

# "Simple" Diagramme
import matplotlib.pyplot as plt

# Komplexe Diagramme (exportierbar in html)
import seaborn as sns

# Einfache Klasse zum herunterladen von Google Drive Dateien
from google_drive_downloader import GoogleDriveDownloader as gdd

# Miscellaneous operating system interfaces
import os as os

# Basic date and time types
import datetime as dt

Wann wird das Chatlog gefetched

In [3]:
# Fetche das aktuelle Chatlog, wenn das aktuell gefetchte Chatlog älter als zwei Tage ist, oder das Chatlog nicht vorhanden ist

# Fall 2: Chatlog ist nicht vorhanden:
if os.path.isfile('./Chatlog.txt'):
    FileDoesntExist=False
    print("File exists")
else:
    FileDoesntExist=True
    print("File doesn't exist")

# Fall 1: Timestamp des Chatlogs ist älter als zwei Tage:
if not FileDoesntExist:
    two_days_ago = dt.datetime.now() - dt.timedelta(days=2)
    filetime = dt.datetime.fromtimestamp(os.path.getctime('./Chatlog.txt'))

    if filetime < two_days_ago:
        FetchFileDate=True
        print("File is more than two days old, new file will be fetched.")

    elif filetime > two_days_ago:
        FetchFileDate=False
        print("File is not more than two days old, new file won't be fetched.")

File exists
File is not more than two days old, new file won't be fetched.


Fetchen des aktuellen Chatlogs

In [4]:
# ToDo: Definiertes ausführen des Fetchvorgangs, abhängig davon wie alt das File ist
if FetchFileDate or FileDoesntExist:
    gdd.download_file_from_google_drive(file_id='1yvKE3iHCT8Rjm_iUkSJzXHVbiVhC3xxL',
                                    dest_path='./Chatlog.txt',
                                    unzip=False,
                                    overwrite=True)
else:
    print("New file won't be fetched, since the file isn't too old.")

New file won't be fetched, since the file isn't too old.


Definition des Chat-Dataframes

In [55]:
# Import der History.txt. Getrennt in 2 Spalten durch das ] Zeichen am Ende des Timestamps
# Spaltennamen als Timestamp und name_and_message definiert. Keine Kopfzeile.
# Text und Name muss noch getrennt werden. 
chat = pd.read_csv('Chatlog.txt', delimiter = ']', names = ['timestamp', 'name_and_message'], header = None)

# Leere Spalte für messages eingefügt
chat['message'] = ''

# Durchläuft den Chat von hinten nach vorne und prüft ob name_and_message ein missing value ist. Wenn ja, befindet sich die Nachricht im Timestamp-Feld und wird in das
# message Feld des vorherigen Eintrags eingetragen und dazu summiert. So Befindet sich die gesamte Nachricht beim zugehörigen Timestamp

for row_number in reversed(range(len(chat))):
    if pd.isna(chat.name_and_message[row_number]):
        chat['message'][row_number-1] =  chat.timestamp[row_number] + chat['message'][row_number]

# Die Zeilen mit missing values wurden so bearbeitet und können entfernt werden.

chat.dropna(inplace = True)

# Aufbereiten der timestamp-column zur Umwandlung des Objekts in ein Datetime-Objekt
# Entfernen des [ am Start des Timestamps

#chat.timestamp.replace(regex = True, inplace = True, to_replace = r'\[', value = r'')
chat.timestamp.replace(regex = True, inplace = True, to_replace = r'\u200e', value = r'')

# Umwandlung zu Datetime

#chat['timestamp'] = pd.to_datetime(chat['timestamp'], format = '[%d.%m.%y, %H:%M:%S')

chat.tail()

,timestamp,name_and_message,message
92675,"[27.04.19, 18:51:49",Tim Ackermann: ‎Bild weggelassen,
92676,"[27.04.19, 19:11:07",Yannic Büchele: ?,
92677,"[27.04.19, 19:22:46",emptytides: .,
92678,"[27.04.19, 19:26:15",Yannic Büchele: Echt so,
92679,"[27.04.19, 19:26:18",Yannic Büchele: Richtig was los hier,
